# Demonstration RAG-Eval


Quick test: see if things work as planned

In [1]:
from rag_eval.protocol import LLMClient
from rag_eval.client import AzureClient
from rag_eval.components import ClaimExtractor, EntailmentJudge
from rag_eval.data_models import EvalContainer, ClaimVerdict
from rag_eval.evaluators import AnswerPrecisionEvaluator, RetrievalPrecisionEvaluator
from rag_eval.coordinator import RAGEvaluator
from rag_eval.helpers import _safe_json
import json
from data.rag_eval_samples import test_examples
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

c:\Users\ANDERBE\AI-Agent-POC-git\AI-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


# Test Agent

Check if agent module actually works.

In [3]:
# load agent and pipeline for indexing docs
import agent.main as main
from rag.indexing import pdf_files, index_pipeline

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [4]:
index_pipeline.run({"converter": {"sources": pdf_files}})

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
Batches: 100%|██████████| 2/2 [00:00<00:00,  2.10it/s]


{'writer': {'documents_written': 51}}

In [5]:
main.food_agent(user_input="hi")


▶️ Step 1: thinking…
🍕 Fooder: Hello! I'm Fooder, your food ordering agent. 🍔🍕🥤

Would you like to see the available food options or place an order today? Let me know how I can help you!


## RAG-Loop
Loop which passes questions to agent and extracts generated answers alongside retrieved contexts (plain text). Combines them with ground truth and query to form `EvalContainer`-object.

In [5]:
# quick test of my query_gt_extractor function
from src.rag_eval.helpers import query_gt_extractor

queries, gt = query_gt_extractor(path="data/agent_eval_questions.json", tokenize=False)
print(gt)


['2010', 'La cucina povera', 'Auguste\u202fEscoffier', 'Jerry\u202fThomas', '1920\u202f–\u202f1933', 'Orecchiette', 'Tarte\u202fTatin', '8,50\u202f€', 'Horchata', 'Secondo', 'Béchamel', 'Reis', 'Mezcal', 'Churros', '2,00\u202f€', 'Risotto\u202fal\u202fTartufo\u202fNero\u202fdel\u202fMastino', 'Mole', 'Camembert', 'Tiramisù\u202fClassico\u202f&\u202fCoda\u202fdi\u202fVolpe', 'Rindfleisch', 'Madrid', 'R', '1969', '7', 'Ernest\xa0Hemingway', 'Mais, Bohnen, Chili, Kürbis, Kakao; UNESCO‑Jahr\xa02010', 'Mexikanische Küche\xa02010; Gastronomisches Mahl der Franzosen\xa02010', 'Parmigiano\u202fReggiano, Mozzarella\u202fdi\u202fBufala; Camembert, Roquefort', 'Reis (Risotto) für Norditalien; Olivenöl für die Provence', 'Prosecco Extra Dry del Cucciolo\u202f–\u202f50,00\u202f€; Vin Rouge Français\u202f(0,75\u202fl)\u202f–\u202f18,00\u202f€', 'Martini, Manhattan, Daiquiri; Negroni', 'Elote – gegrillter Mais mit Limette und Käse; Caprese (Mozzarella, Tomate, Basilikum)', 'Béchamel; Filetto\u202fdi\

This code cell creates a RAG-loop for the agent. It:  
- feeds the questions from the test battery to the agent  
- extracts the generated answer and retrieved context from the RAG-call
- returns a list of `EvalContainer`-objects storing question, ground truth, generated answer and retrieved context

It thus lays the foundation for the `RAGEevaluator`. 

In [7]:
from src.rag_eval.eval_loop import rag_loop_agent
result = rag_loop_agent()


▶️ Step 1: thinking…

▶️ Step 1: thinking…

▶️ Step 1: thinking…

▶️ Step 1: thinking…
TOOL CALL ▶ execute_agentic_rag {"question":"Wer gilt laut Cocktail-Dokument als Vater der modernen Barkultur?"}
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.15it/s]


(['Laut dem Cocktail-Dokument gilt Jerry Thomas als Vater der modernen Barkultur.'], [['erlebte die Barkultur eine Renaissance. Mixologen begannen, sich wieder\nauf Qualitaet, Balance und handwerkliche Praezision zu konzentrieren. Die "Craft\nCocktail"-Bewegung setzte sich gegen Fertigmischungen und Sirup-Cocktails durch und brachte\nKlassiker zurueck auf die Karte. Barkeeper wurden zu Kuenstlern - mit Kraeutern, Rauch, Infusionen\nund selbstgemachten Zutaten kreierten sie neue Geschmackserlebnisse. Weltweit entstanden Bars\nmit thematischen ', 'Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktails auch als kulturelle Artefakte verstanden: Der Negroni\nstammt aus Italien, der Caipirinha aus Brasilien, der Mojito aus Kuba. Trends wie alkoholfreie\nCoc

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.12it/s]


(['Die Prohibition in den USA dauerte laut Cocktail-Dokument von 1920 bis 1933.'], [['Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Geschichte des Cocktail-Mixings\nUeber das Land\nDie Geschichte des Cocktail-Mixings ist eng mit gesellschaftlichen, wirtschaftlichen und politischen\nEntwicklungen verbunden. Erste schriftliche Erwaehnungen des Begriffs "Cocktail" tauchten Anfang\ndes 19. Jahrhunderts in den USA auf - definiert als "stimulierendes Getraenk aus Spirituosen,\nZucker, Wasser und Bitters". Das sogenannte "Golden Age of Cocktails" begann in der Mitte ', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktails auch als kulturelle Artefakte verstanden: Der Negroni\nstammt aus Italien, der Caipirinha aus Brasilien, der Mojito aus Kuba. Trends wie alkoholfreie\nCocktails ("Mocktails"), lokale

...

(['Die drei Bestandteile des Carne Asada Taco aus dem Tönis‑Tacos‑Menü sind:  \n**Rindfleisch, Zwiebeln, Koriander.**\n\nDie vollständige Bezeichnung der Pizza «Doggish Special» lautet:  \n**Doggish Special (Tomate, Mozzarella, Rinderhack, Speck, Zwiebeln).**'], [['Tönis Tacos\nVorspeisen\nGuacamole mit Tortilla-Chips - 5,50 €\nQuesadillas mit Käse und Jalapeños - 6,00 €\nMexikanische Bohnen-Suppe - 4,50 €\nElote – gegrillter Mais mit Limette und Käse - 5,00 €\nTacos\nCarne Asada Taco (Rindfleisch, Zwiebeln, Koriander) - 3,50 €\nPollo Taco (gegrilltes Hähnchen, Pico de Gallo) - 3,50 €\nVeggie Taco (gegrilltes Gemüse, Avocado) - ', "Anton's Pizza - Doggish Delights\nVorspeisen\nBruschetta mit Tomaten und Basilikum - 5,50 €\nKnoblauchbrot mit Kräuterbutter - 4,00 €\nCaprese (Mozzarella, Tomate, Basilikum) - 6,50 €\nFrittierte Mozzarella-Sticks - 6,00 €\nPizza\nMargherita (Tomate, Mozzarella, Basilikum) - 8,50 €\nDoggish Special (Tomate, Mozzarella, Rinderhack, Speck, Zwiebeln) - 11,50 €\

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.69it/s]


(['Im Tönis-Tacos-Menü gibt es die Bohnenbeilage "Refried Beans (zerdrückte Bohnen)" für 3,00\u202f€. Ja, diese Beilage verwendet mit Bohnen eine der „heiligen Fünf“ Zutaten der mexikanischen Küche (Mais, Bohnen, Chili, Kürbis, Kakao).'], [['Tönis Tacos\nVorspeisen\nGuacamole mit Tortilla-Chips - 5,50 €\nQuesadillas mit Käse und Jalapeños - 6,00 €\nMexikanische Bohnen-Suppe - 4,50 €\nElote – gegrillter Mais mit Limette und Käse - 5,00 €\nTacos\nCarne Asada Taco (Rindfleisch, Zwiebeln, Koriander) - 3,50 €\nPollo Taco (gegrilltes Hähnchen, Pico de Gallo) - 3,50 €\nVeggie Taco (gegrilltes Gemüse, Avocado) - ', '€\nRefried Beans (zerdrückte Bohnen) - 3,00 €\nSalsa Trio (Mild, Medium, Scharf) - 4,00 €\nGetränke\nAgua Fresca (verschiedene Sorten) - 3,00 €\nMexikanisches Bier (0,33 l) - 4,00 €\nMargarita (klassisch) - 7,00 €\nHorchata (Reis-Getränk) - 3,50 €\nEspresso - 2,00 €\nCafé de Olla (mexikanischer Kaffee) - 3,00 €', '3,00 €\nFish Taco (gebratener Fisch, Krautsalat) - 4,00 €\nBurritos\

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.51it/s]


(['Das Meeresfrüchte-Risotto heißt auf der Speisekarte von Anton’s Pizza:\n\n**Meeresfrüchte-Risotto**'], [["Mozzarella, Paprika, Zucchini, Oliven) - 10,00 €\nPasta\nSpaghetti Bolognese - 9,50 €\nTagliatelle mit Pesto und Pinienkernen - 10,00 €\nPenne Arrabiata (pikant) - 8,50 €\nLasagne al Forno - 11,00 €\x0cAnton's Pizza - Doggish Delights\nRisotto\nRisotto ai Funghi (mit Pilzen) - 11,50 €\nMeeresfrüchte-Risotto - 14,00 €\nRisotto Primavera (Gemüse der Saison) - 12,00 €\nGetränke\nMineralwasser (0,5 ", "Anton's Pizza - Doggish Delights\nVorspeisen\nBruschetta mit Tomaten und Basilikum - 5,50 €\nKnoblauchbrot mit Kräuterbutter - 4,00 €\nCaprese (Mozzarella, Tomate, Basilikum) - 6,50 €\nFrittierte Mozzarella-Sticks - 6,00 €\nPizza\nMargherita (Tomate, Mozzarella, Basilikum) - 8,50 €\nDoggish Special (Tomate, Mozzarella, Rinderhack, Speck, Zwiebeln) - 11,50 €\nFunghi (Tomate, Mozzarella, Champignons) - 9,50 €\nVeggie Delight (Tomate, ", 'Kueche ist weltweit bekannt und wird fuer ihre Ei

In [8]:
# show eval containers
print(result)

[EvalContainer(query='Wer gilt laut Cocktail‑Dokument als Vater der modernen Barkultur?', ground_truth_answer='Jerry\u202fThomas', generated_answer='Laut dem Cocktail-Dokument gilt Jerry Thomas als Vater der modernen Barkultur.', retrieved_texts=['erlebte die Barkultur eine Renaissance. Mixologen begannen, sich wieder\nauf Qualitaet, Balance und handwerkliche Praezision zu konzentrieren. Die "Craft\nCocktail"-Bewegung setzte sich gegen Fertigmischungen und Sirup-Cocktails durch und brachte\nKlassiker zurueck auf die Karte. Barkeeper wurden zu Kuenstlern - mit Kraeutern, Rauch, Infusionen\nund selbstgemachten Zutaten kreierten sie neue Geschmackserlebnisse. Weltweit entstanden Bars\nmit thematischen ', 'Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cock

## Evaluation of Agentic RAG with RAGEvaluator


In [9]:
# initiate llm 
api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


# run evaluator
coord = RAGEvaluator(llm_extractor=llm, llm_judge=llm)
data = result[0:2]


for i, res in enumerate(data, start=1):
    print(f"EXAMPLE {i}")
    results = coord.evaluate_all(res)
    print(json.dumps(results, indent=2))


EXAMPLE 1
{
  "answer": {
    "gt_claims": [
      "Jerry Thomas is a person."
    ],
    "gen_claims": [
      "Das Cocktail-Dokument behauptet, dass Jerry Thomas als Vater der modernen Barkultur gilt."
    ],
    "gt_claim_recall": 1.0,
    "hallucination_precision": 0.0,
    "gt_claim_verdicts": [
      {
        "claim": "Jerry Thomas is a person.",
        "verdict": "supported",
        "rationale": "the evidence refers to jerry thomas as the 'vater der modernen barkultur' (father of modern bar culture), which implies that jerry thomas is a person. therefore, the claim is supported.",
        "doc_indices": null
      }
    ],
    "gen_claim_verdicts": [
      {
        "claim": "Das Cocktail-Dokument behauptet, dass Jerry Thomas als Vater der modernen Barkultur gilt.",
        "verdict": "not_enough_info",
        "rationale": "the evidence only provides the name 'jerry thomas' without any information about his association with the modern bar culture or any claims made in the 'c